In [1]:
import pandas as pd

df = pd.read_csv("MovieReview.csv")
display(df.head())
print(df.shape)

df = df.drop('sentiment', axis=1)

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...


(25000, 2)


In [3]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp311-cp311-macosx_10_9_x86_64.whl.metadata (40 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp311-cp311-macosx_10_9_x86_64.whl (287 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download()
stop_words = stopwords.words('english')

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df.review = df.review.apply(lambda x :preprocess_sentence(x))
df.head()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [5]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df.review)

2024-12-10 11:06:39.743997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words

In [ ]:
import pickle
with open('vocab_infor.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([word2idx, idx2word, vocab_size], f)

In [ ]:
import hickle as hkl
# write variables to filename [a,b,c can be of any size]
filename = 'vocab_info'
hkl.dump([word2idx, idx2word, vocab_size], filename)

# load variables from filename
word2idx, idx2word, vocab_size = hkl.load(filename)

In [7]:
import numpy as np


def sentenceToData(tokens, WINDOW_SIZE):
    window = np.concatenate((np.arange(-WINDOW_SIZE,0),np.arange(1,WINDOW_SIZE+1)))
    X,Y=([],[])
    for word_index, word in enumerate(tokens) :
        if ((word_index - WINDOW_SIZE >= 0) and (word_index + WINDOW_SIZE <= len(tokens) - 1)) :
            X.append(word)
            Y.append([tokens[word_index-i] for i in window])
    return X, Y


WINDOW_SIZE = 5

X, Y = ([], [])
for review in df.review:
    for sentence in review.split("."):
        word_list = tokenizer.texts_to_sequences([sentence])[0]
        if len(word_list) >= WINDOW_SIZE:
            Y1, X1 = sentenceToData(word_list, WINDOW_SIZE//2)
            X.extend(X1)
            Y.extend(Y1)
    
X = np.array(X).astype(int)
y = np.array(Y).astype(int).reshape([-1,1])

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D

embedding_dim = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, batch_size = 128, epochs=50)

Epoch 1/50
12163/12163 [==============================] - 515s 42ms/step - loss: 7.6543 - accuracy: 0.0313
Epoch 2/50
12163/12163 [==============================] - 469s 39ms/step - loss: 6.9537 - accuracy: 0.0581
Epoch 3/50
12163/12163 [==============================] - 412s 34ms/step - loss: 6.5205 - accuracy: 0.0754
Epoch 4/50
12163/12163 [==============================] - 438s 36ms/step - loss: 6.1902 - accuracy: 0.0886
Epoch 5/50
12163/12163 [==============================] - 521s 43ms/step - loss: 5.9204 - accuracy: 0.1005
Epoch 6/50
12163/12163 [==============================] - 527s 43ms/step - loss: 5.6976 - accuracy: 0.1111
Epoch 7/50
12163/12163 [==============================] - 447s 37ms/step - loss: 5.5147 - accuracy: 0.1208
Epoch 8/50
12163/12163 [==============================] - 413s 34ms/step - loss: 5.3656 - accuracy: 0.1303
Epoch 9/50
12163/12163 [==============================] - 412s 34ms/step - loss: 5.2435 - accuracy: 0.1388
Epoch 10/50
12163/12163 [============

In [ ]:
model.save("word2vec.h5") 